In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [ ]:
data = pd.read_csv('mimic CA.csv')
data

,stay_id,subject_id,gender,anchor_age,starttime,charttime,heart_rate,sbp_ni,dbp_ni,resp_rate,temperature,spo2
0,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 18:07:00,114.0,85.0,60.0,18.0,37.3,96.0
1,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 18:18:00,111.0,85.0,60.0,19.0,37.3,96.0
2,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 18:32:00,108.0,85.0,60.0,20.0,37.3,95.0
3,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 20:00:00,111.0,85.0,60.0,18.0,37.3,96.0
4,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 20:15:00,111.0,85.0,60.0,22.0,37.3,96.0
...,...,...,...,...,...,...,...,...,...,...,...,...
68968,39992578,19774838,M,69,2187-09-15 10:00:00,2187-09-15 06:40:00,85.0,88.0,33.0,11.0,37.5,99.0
68969,39992578,19774838,M,69,2187-09-15 10:00:00,2187-09-15 07:00:00,95.0,88.0,33.0,16.0,37.7,99.0
68970,39992578,19774838,M,69,2187-09-15 10:00:00,2187-09-15 08:00:00,100.0,88.0,33.0,18.0,37.8,99.0
68971,39992578,19774838,M,69,2187-09-15 10:00:00,2187-09-15 09:00:00,104.0,88.0,33.0,22.0,37.8,98.0


In [ ]:
print(data['stay_id'].nunique())

479


In [ ]:
# 將時間欄位轉換成 datetime 格式
data['starttime'] = pd.to_datetime(data['starttime'])
data['charttime'] = pd.to_datetime(data['charttime'])

In [ ]:
# 將每位患者分成一人一組
group_data = data.groupby(['stay_id'])
group_data

In [ ]:
# 去除每位患者心臟驟停前48小時更之前的資料
def delete_hours(group):
    time_column = 'starttime'
    target_column = 'charttime'

    time_threshold = group[time_column] - timedelta(hours=48)
    group = group[group[target_column] >= time_threshold]

    return group

groups = group_data.apply(delete_hours)
groups

stay_id  subject_id gender  anchor_age           starttime  \
stay_id                                                                       
30031755 107    30031755    19867017      F          74 2124-04-24 17:00:00   
         108    30031755    19867017      F          74 2124-04-24 17:00:00   
         109    30031755    19867017      F          74 2124-04-24 17:00:00   
         110    30031755    19867017      F          74 2124-04-24 17:00:00   
         111    30031755    19867017      F          74 2124-04-24 17:00:00   
...                  ...         ...    ...         ...                 ...   
39992578 68968  39992578    19774838      M          69 2187-09-15 10:00:00   
         68969  39992578    19774838      M          69 2187-09-15 10:00:00   
         68970  39992578    19774838      M          69 2187-09-15 10:00:00   
         68971  39992578    19774838      M          69 2187-09-15 10:00:00   
         68972  39992578    19774838      M          69 2187-09-15 10:00:00   

                         charttime  heart_rate  sbp_ni  dbp_ni  resp_rate  \
stay_id                                                                     
30031755 107   2124-04-22 17:00:00        94.0    95.0    56.0       11.0   
         108   2124-04-22 17:11:00        96.0    90.0    56.0       14.0   
         109   2124-04-22 19:00:00        98.0    84.0    57.0       17.0   
         110   2124-04-22 20:00:00        94.0    88.0    29.0       17.0   
         111   2124-04-22 20:02:00        98.0    83.0    29.0       16.0   
...                            ...         ...     ...     ...        ...   
39992578 68968 2187-09-15 06:40:00        85.0    88.0    33.0       11.0   
         68969 2187-09-15 07:00:00        95.0    88.0    33.0       16.0   
         68970 2187-09-15 08:00:00       100.0    88.0    33.0       18.0   
         68971 2187-09-15 09:00:00       104.0    88.0    33.0       22.0   
         68972 2187-09-15 10:00:00        90.0    88.0    33.0       18.0   

                temperature  spo2  
stay_id                            
30031755 107           35.3  93.0  
         108           35.3  94.0  
         109           35.3  99.0  
         110           35.2  98.0  
         111           35.2  97.0  
...                     ...   ...  
39992578 68968         37.5  99.0  
         68969         37.7  99.0  
         68970         37.8  99.0  
         68971         37.8  98.0  
         68972         37.8  98.0  

[22705 rows x 12 columns]

In [ ]:
print(groups['stay_id'].nunique())

479


In [ ]:
groups.reset_index(drop=True, inplace=True)
groups

,stay_id,subject_id,gender,anchor_age,starttime,charttime,heart_rate,sbp_ni,dbp_ni,resp_rate,temperature,spo2
0,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-22 17:00:00,94.0,95.0,56.0,11.0,35.3,93.0
1,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-22 17:11:00,96.0,90.0,56.0,14.0,35.3,94.0
2,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-22 19:00:00,98.0,84.0,57.0,17.0,35.3,99.0
3,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-22 20:00:00,94.0,88.0,29.0,17.0,35.2,98.0
4,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-22 20:02:00,98.0,83.0,29.0,16.0,35.2,97.0
...,...,...,...,...,...,...,...,...,...,...,...,...
22700,39992578,19774838,M,69,2187-09-15 10:00:00,2187-09-15 06:40:00,85.0,88.0,33.0,11.0,37.5,99.0
22701,39992578,19774838,M,69,2187-09-15 10:00:00,2187-09-15 07:00:00,95.0,88.0,33.0,16.0,37.7,99.0
22702,39992578,19774838,M,69,2187-09-15 10:00:00,2187-09-15 08:00:00,100.0,88.0,33.0,18.0,37.8,99.0
22703,39992578,19774838,M,69,2187-09-15 10:00:00,2187-09-15 09:00:00,104.0,88.0,33.0,22.0,37.8,98.0


In [ ]:
# 正樣本
positive = pd.DataFrame()

groupss = groups.groupby('stay_id')

for group_id, group_df in groupss:
    # 選擇每位患者CA前一小時到前五小時的資料
    end_time_df1 = group_df['starttime']- timedelta(hours=1)
    start_time_df1 = group_df['starttime'] - timedelta(hours=5)

    group_df1 = group_df[(group_df['charttime'] >= start_time_df1) & (group_df['charttime'] <= end_time_df1)]

    for index, row in group_df1.iterrows():
        # 將每位患者此區間的每一筆資料，依照對應的時間將資料選取出來(當筆、前四小時、前八小時到前二十小時)
        end_time_df2 = row['charttime']
        time_ranges = [20, 16, 12, 8, 4, 0]

        for hours in time_ranges:
            start_time_df2 = end_time_df2 - timedelta(hours=hours)

            # 選取後將資料新增到新的dataframe
            data_point = group_df[(group_df['charttime'] >= start_time_df2) & (group_df['charttime'] <= start_time_df2)]

            # 若沒有抓到資料，就選擇正負三十分鐘內的隨機一筆資料
            if data_point.empty:
                minutes = pd.to_timedelta(30, unit='m')
                data_point = group_df[(group_df['charttime'] >= (start_time_df2-minutes)) & (group_df['charttime'] <= (start_time_df2-minutes))].head(1)

            # 每位患者的每筆樣本若不到六個資料點，就用0來補值
            if data_point.empty:
                data_point = pd.DataFrame({'stay_id': [group_id] * 1,
                          'starttime': [np.nan] * 1,
                          'charttime': [np.nan] * 1,
                          'heart_rate': [0] * 1,
                          'sbp_ni': [0] * 1,
                          'dbp_ni': [0] * 1,
                          'resp_rate': [0] * 1,
                          'temperature': [0] * 1,
                          'spo2': [0] * 1})

            positive = pd.concat([positive, data_point])

positive.reset_index(drop=True, inplace=True)
positive1 = positive[['stay_id', 'starttime', 'charttime', 'heart_rate', 'sbp_ni', 'dbp_ni', 'resp_rate', 'temperature' ,'spo2']]
positive1

,stay_id,starttime,charttime,heart_rate,sbp_ni,dbp_ni,resp_rate,temperature,spo2
0,30031755,2124-04-24 17:00:00,2124-04-23 16:00:00,98.0,84.0,47.0,14.0,36.4,95.0
1,30031755,2124-04-24 17:00:00,2124-04-23 20:00:00,97.0,75.0,48.0,11.0,36.2,97.0
2,30031755,2124-04-24 17:00:00,2124-04-24 00:00:00,87.0,78.0,48.0,16.0,36.0,96.0
3,30031755,2124-04-24 17:00:00,2124-04-24 04:00:00,91.0,74.0,44.0,16.0,35.9,89.0
4,30031755,2124-04-24 17:00:00,2124-04-24 08:00:00,92.0,81.0,47.0,15.0,35.8,91.0
...,...,...,...,...,...,...,...,...,...
20767,39992578,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0
20768,39992578,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0
20769,39992578,2187-09-15 10:00:00,2187-09-15 01:00:00,80.0,86.0,37.0,18.0,36.5,100.0
20770,39992578,2187-09-15 10:00:00,2187-09-15 05:00:00,83.0,87.0,35.0,16.0,36.6,100.0


In [ ]:
print(positive1['stay_id'].nunique())

418


In [ ]:
# 負樣本

def select_data(group):
    time_ranges = [49, 45, 41, 37, 32, 29, 25, 21, 17, 13, 9, 5]

    selected_data = []

    for hours in time_ranges:
        end_time = group['starttime'] - pd.to_timedelta(hours, unit='h')
        selected_group = group[(group['charttime'] >= end_time) & (group['charttime'] <= end_time)]

        if selected_group.empty:
            minutes = pd.to_timedelta(30, unit='m')
            selected_group = group[(group['charttime'] >= (end_time - minutes)) & (group['charttime'] <= (end_time + minutes))].head(1)

        selected_data.append(selected_group)

    return pd.concat(selected_data, ignore_index=True)

negative = groups.groupby('stay_id').apply(select_data)
negative.reset_index(drop=True, inplace=True)
negative = negative[['stay_id', 'starttime', 'charttime', 'heart_rate', 'sbp_ni', 'dbp_ni', 'resp_rate', 'temperature' ,'spo2']]
negative

,stay_id,starttime,charttime,heart_rate,sbp_ni,dbp_ni,resp_rate,temperature,spo2
0,30031755,2124-04-24 17:00:00,2124-04-22 20:00:00,94.0,88.0,29.0,17.0,35.2,98.0
1,30031755,2124-04-24 17:00:00,2124-04-23 00:00:00,99.0,86.0,48.0,12.0,35.1,97.0
2,30031755,2124-04-24 17:00:00,2124-04-23 04:00:00,105.0,96.0,56.0,18.0,35.2,97.0
3,30031755,2124-04-24 17:00:00,2124-04-23 09:00:00,106.0,84.0,52.0,16.0,35.9,95.0
4,30031755,2124-04-24 17:00:00,2124-04-23 12:00:00,87.0,82.0,46.0,15.0,35.8,91.0
...,...,...,...,...,...,...,...,...,...
2752,39950840,2147-06-22 18:30:00,2147-06-22 05:00:00,74.0,80.0,53.0,26.0,36.6,58.0
2753,39950840,2147-06-22 18:30:00,2147-06-22 09:00:00,73.0,82.0,49.0,29.0,37.3,84.0
2754,39950840,2147-06-22 18:30:00,2147-06-22 13:00:00,79.0,83.0,46.0,30.0,37.1,98.0
2755,39992578,2187-09-15 10:00:00,2187-09-15 01:00:00,80.0,86.0,37.0,18.0,36.5,100.0


In [ ]:
# 考慮患者剛入院，不會一次跳出六筆測量資料
# 在每位患者一開始的資料新增5行未知的資料(在features中填入0或NaN)，使input維持特定shape(6,6)
new_rows = []

for group, group_data in negative.groupby('stay_id'):

    front_rows = pd.DataFrame({'stay_id': [group] * 5, 'starttime': [None] * 5, 'charttime': [None] * 5,
                               'heart_rate': [0] * 5, 'sbp_ni': [0] * 5, 'dbp_ni': [0] * 5,
                               'resp_rate': [0] * 5, 'temperature': [0] * 5, 'spo2': [0] * 5})

    new_rows.extend([front_rows, group_data])

negative1 = pd.concat(new_rows, ignore_index=True)
negative1

,stay_id,starttime,charttime,heart_rate,sbp_ni,dbp_ni,resp_rate,temperature,spo2
0,30031755,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0
1,30031755,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0
2,30031755,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0
3,30031755,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0
4,30031755,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
4567,39992578,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0
4568,39992578,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0
4569,39992578,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0
4570,39992578,2187-09-15 10:00:00,2187-09-15 01:00:00,80.0,86.0,37.0,18.0,36.5,100.0


In [ ]:
# 存放正樣本的dataframe
pos_sampless = pd.DataFrame()

pos_samples = []

order = 1

window_size = 6

for group, group_data in positive1.groupby('stay_id'):
    group_len = len(group_data)

    for i in range(0, group_len, 6):
        pos_sample = group_data.iloc[i:i + window_size]
        pos_sample = pos_sample.copy()
        pos_sample['order'] = order
        pos_samples.append(pos_sample)
        order += 1

pos_sampless = pd.concat(pos_samples)
pos_sampless.reset_index(drop=True, inplace=True)
pos_sampless

,stay_id,starttime,charttime,heart_rate,sbp_ni,dbp_ni,resp_rate,temperature,spo2,order
0,30031755,2124-04-24 17:00:00,2124-04-23 16:00:00,98.0,84.0,47.0,14.0,36.4,95.0,1
1,30031755,2124-04-24 17:00:00,2124-04-23 20:00:00,97.0,75.0,48.0,11.0,36.2,97.0,1
2,30031755,2124-04-24 17:00:00,2124-04-24 00:00:00,87.0,78.0,48.0,16.0,36.0,96.0,1
3,30031755,2124-04-24 17:00:00,2124-04-24 04:00:00,91.0,74.0,44.0,16.0,35.9,89.0,1
4,30031755,2124-04-24 17:00:00,2124-04-24 08:00:00,92.0,81.0,47.0,15.0,35.8,91.0,1
...,...,...,...,...,...,...,...,...,...,...
20767,39992578,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0,3462
20768,39992578,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0,3462
20769,39992578,2187-09-15 10:00:00,2187-09-15 01:00:00,80.0,86.0,37.0,18.0,36.5,100.0,3462
20770,39992578,2187-09-15 10:00:00,2187-09-15 05:00:00,83.0,87.0,35.0,16.0,36.6,100.0,3462


In [ ]:
# 存放負樣本的dataframe
neg_sampless = pd.DataFrame()

neg_samples = []

order = 1

window_size = 6

for group, group_data in negative1.groupby('stay_id'):
    group_len = len(group_data)

    for i in range(group_len - window_size + 1):
        neg_sample = group_data.iloc[i:i + window_size]
        neg_sample = neg_sample.copy()
        neg_sample['order'] = order
        neg_samples.append(neg_sample)
        order += 1

    if group_len < window_size:
        neg_sample = group_data.copy()
        neg_sample['order'] = order
        neg_samples.append(neg_sample)
        order += 1

neg_sampless = pd.concat(neg_samples)
neg_sampless.reset_index(drop=True, inplace=True)
neg_sampless

,stay_id,starttime,charttime,heart_rate,sbp_ni,dbp_ni,resp_rate,temperature,spo2,order
0,30031755,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0,1
1,30031755,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0,1
2,30031755,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0,1
3,30031755,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0,1
4,30031755,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...
16537,39992578,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0,2757
16538,39992578,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0,2757
16539,39992578,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0,2757
16540,39992578,2187-09-15 10:00:00,2187-09-15 01:00:00,80.0,86.0,37.0,18.0,36.5,100.0,2757


In [ ]:
pos_sampless = pos_sampless.replace(0, np.nan)
neg_sampless = neg_sampless.replace(0, np.nan)

In [ ]:
# 計算正樣本中的各vital signs平均
print(pos_sampless['heart_rate'].dropna().mean())
print(pos_sampless['heart_rate'].dropna().std())
print('-'*30)
print(pos_sampless['sbp_ni'].dropna().mean())
print(pos_sampless['sbp_ni'].dropna().std())
print('-'*30)
print(pos_sampless['dbp_ni'].dropna().mean())
print(pos_sampless['dbp_ni'].dropna().std())
print('-'*30)
print(pos_sampless['resp_rate'].dropna().mean())
print(pos_sampless['resp_rate'].dropna().std())
print('-'*30)
print(pos_sampless['temperature'].dropna().mean())
print(pos_sampless['temperature'].dropna().std())
print('-'*30)
print(pos_sampless['spo2'].dropna().mean())
print(pos_sampless['spo2'].dropna().std())

93.23456790123457
22.321661776466335
------------------------------
107.03880070546737
23.91783386037138
------------------------------
60.84440525181266
16.153474226841876
------------------------------
22.10944542426024
6.772908750368865
------------------------------
36.63689006466842
1.1277611720619043
------------------------------
95.07720948461689
7.271841747683157


In [ ]:
# 計算負樣本中的各vital signs平均
print(neg_sampless['heart_rate'].dropna().mean())
print(neg_sampless['heart_rate'].dropna().std())
print('-'*30)
print(neg_sampless['sbp_ni'].dropna().mean())
print(neg_sampless['sbp_ni'].dropna().std())
print('-'*30)
print(neg_sampless['dbp_ni'].dropna().mean())
print(neg_sampless['dbp_ni'].dropna().std())
print('-'*30)
print(neg_sampless['resp_rate'].dropna().mean())
print(neg_sampless['resp_rate'].dropna().std())
print('-'*30)
print(neg_sampless['temperature'].dropna().mean())
print(neg_sampless['temperature'].dropna().std())
print('-'*30)
print(neg_sampless['spo2'].dropna().mean())
print(neg_sampless['spo2'].dropna().std())

90.13257607866407
19.526591608412463
------------------------------
109.565991353734
22.406952919641128
------------------------------
61.07951174027295
14.759417503801137
------------------------------
21.385691277443417
6.248696742301505
------------------------------
36.80136475375115
0.8885718052724718
------------------------------
96.27896922946512
5.049306834991078


In [ ]:
from scipy.stats import levene

# 正負樣本的各vital sign作統計比較
# 變異數
hstatistic, hp_value = levene(pos_sampless['heart_rate'].dropna().tolist(), neg_sampless['heart_rate'].dropna().tolist())
sstatistic, sp_value = levene(pos_sampless['sbp_ni'].dropna().tolist(), neg_sampless['sbp_ni'].dropna().tolist())
dstatistic, dp_value = levene(pos_sampless['dbp_ni'].dropna().tolist(), neg_sampless['dbp_ni'].dropna().tolist())
rstatistic, rp_value = levene(pos_sampless['resp_rate'].dropna().tolist(), neg_sampless['resp_rate'].dropna().tolist())
tstatistic, tp_value = levene(pos_sampless['temperature'].dropna().tolist(), neg_sampless['temperature'].dropna().tolist())
ostatistic, op_value = levene(pos_sampless['spo2'].dropna().tolist(), neg_sampless['spo2'].dropna().tolist())


print('統計量：%.2f'%hstatistic)
print('p-value:%.5f'%hp_value)
print('-'*30)
print('統計量：%.2f'%sstatistic)
print('p-value:%.5f'%sp_value)
print('-'*30)
print('統計量：%.2f'%dstatistic)
print('p-value:%.5f'%dp_value)
print('-'*30)
print('統計量：%.2f'%rstatistic)
print('p-value:%.5f'%rp_value)
print('-'*30)
print('統計量：%.2f'%tstatistic)
print('p-value:%.5f'%tp_value)
print('-'*30)
print('統計量：%.2f'%ostatistic)
print('p-value:%.5f'%op_value)
print('-'*30)

統計量：203.67
p-value:0.00000
------------------------------
統計量：15.92
p-value:0.00007
------------------------------
統計量：53.44
p-value:0.00000
------------------------------
統計量：52.58
p-value:0.00000
------------------------------
統計量：121.28
p-value:0.00000
------------------------------
統計量：158.19
p-value:0.00000
------------------------------


In [ ]:
from scipy.stats import ttest_ind

# 獨立t test

print(ttest_ind(pos_sampless['heart_rate'].dropna().tolist(), neg_sampless['heart_rate'].dropna().tolist(), equal_var=False, alternative='two-sided'))
print('-'*30)
print(ttest_ind(pos_sampless['sbp_ni'].dropna().tolist(), neg_sampless['sbp_ni'].dropna().tolist(), equal_var=False, alternative='two-sided'))
print('-'*30)
print(ttest_ind(pos_sampless['dbp_ni'].dropna().tolist(), neg_sampless['dbp_ni'].dropna().tolist(), equal_var=False, alternative='two-sided'))
print('-'*30)
print(ttest_ind(pos_sampless['resp_rate'].dropna().tolist(), neg_sampless['resp_rate'].dropna().tolist(), equal_var=False, alternative='two-sided'))
print('-'*30)
print(ttest_ind(pos_sampless['temperature'].dropna().tolist(), neg_sampless['temperature'].dropna().tolist(), equal_var=False, alternative='two-sided'))
print('-'*30)
print(ttest_ind(pos_sampless['spo2'].dropna().tolist(), neg_sampless['spo2'].dropna().tolist(), equal_var=False, alternative='two-sided'))

Ttest_indResult(statistic=10.889833815105167, pvalue=1.5340355384317692e-27)
------------------------------
Ttest_indResult(statistic=-8.047771008446485, pvalue=8.874337137643289e-16)
------------------------------
Ttest_indResult(statistic=-1.1204119480651353, pvalue=0.2625511565915816)
------------------------------
Ttest_indResult(statistic=8.192548474836817, pvalue=2.7027957327434133e-16)
------------------------------
Ttest_indResult(statistic=-11.883988378158936, pvalue=1.8607823214347363e-32)
------------------------------
Ttest_indResult(statistic=-14.024844231280579, pvalue=1.8936171042737844e-44)


In [ ]:
noCA = pd.read_csv('mimic nonCA.csv')
noCA

,label,subject_id,stay_id,gender,anchor_age,charttime,heart_rate,resp_rate,sbp_ni,dbp_ni,spo2,temperature
0,0,10018845,35674354,M,91,2184-10-08 04:21:00,83.0,18.0,157.0,70.0,99.0,36.7
1,0,10018845,35674354,M,91,2184-10-08 04:22:00,75.0,16.0,157.0,70.0,99.0,36.7
2,0,10018845,35674354,M,91,2184-10-08 04:23:00,66.0,15.0,157.0,70.0,98.0,36.7
3,0,10018845,35674354,M,91,2184-10-08 04:31:00,58.0,13.0,146.0,60.0,98.0,36.7
4,0,10018845,35674354,M,91,2184-10-08 05:00:00,49.0,11.0,134.0,49.0,96.0,36.6
...,...,...,...,...,...,...,...,...,...,...,...,...
55764,0,19975710,32714049,F,69,2129-11-26 01:01:00,80.0,16.0,151.0,72.0,94.0,36.7
55765,0,19975710,32714049,F,69,2129-11-26 02:00:00,72.0,13.0,140.0,65.0,96.0,36.7
55766,0,19975710,32714049,F,69,2129-11-26 02:01:00,76.0,14.0,128.0,58.0,95.0,36.7
55767,0,19975710,32714049,F,69,2129-11-26 03:00:00,79.0,14.0,141.0,60.0,94.0,36.7


In [ ]:
print(noCA['stay_id'].nunique())

477


In [ ]:
noCA = noCA.drop(columns=['label'])
noCA = noCA.assign(starttime=np.nan)
noCA

,subject_id,stay_id,gender,anchor_age,charttime,heart_rate,resp_rate,sbp_ni,dbp_ni,spo2,temperature,starttime
0,10018845,35674354,M,91,2184-10-08 04:21:00,83.0,18.0,157.0,70.0,99.0,36.7,NaN
1,10018845,35674354,M,91,2184-10-08 04:22:00,75.0,16.0,157.0,70.0,99.0,36.7,NaN
2,10018845,35674354,M,91,2184-10-08 04:23:00,66.0,15.0,157.0,70.0,98.0,36.7,NaN
3,10018845,35674354,M,91,2184-10-08 04:31:00,58.0,13.0,146.0,60.0,98.0,36.7,NaN
4,10018845,35674354,M,91,2184-10-08 05:00:00,49.0,11.0,134.0,49.0,96.0,36.6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
55764,19975710,32714049,F,69,2129-11-26 01:01:00,80.0,16.0,151.0,72.0,94.0,36.7,NaN
55765,19975710,32714049,F,69,2129-11-26 02:00:00,72.0,13.0,140.0,65.0,96.0,36.7,NaN
55766,19975710,32714049,F,69,2129-11-26 02:01:00,76.0,14.0,128.0,58.0,95.0,36.7,NaN
55767,19975710,32714049,F,69,2129-11-26 03:00:00,79.0,14.0,141.0,60.0,94.0,36.7,NaN


In [ ]:
noCA['charttime'] = pd.to_datetime(noCA['charttime'])

In [ ]:
latest_data = noCA.groupby('stay_id')['charttime'].idxmax()
latest_data = noCA.loc[latest_data]

negative = pd.DataFrame()

time_ranges = [0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44]

for _, latest_row in latest_data.iterrows():

    for time_range in time_ranges:
        start_time = latest_row['charttime'] - timedelta(hours=time_range)
        end_time = latest_row['charttime'] - timedelta(hours=time_range)

        selected_data = noCA.loc[(noCA['stay_id'] == latest_row['stay_id']) &
                               (noCA['charttime'] >= start_time) &
                               (noCA['charttime'] <= end_time)]

        if  selected_data.empty:
            start_time = latest_row['charttime'] - timedelta(hours=time_range) - timedelta(minutes=30)
            end_time = latest_row['charttime'] - timedelta(hours=time_range) + timedelta(minutes=30)
            selected_data = noCA.loc[(noCA['charttime'] >= start_time)
                                                  & (noCA['charttime'] <= end_time).sample(n=1)]

        negative = pd.concat([negative, selected_data])

negative.reset_index(drop=True, inplace=True)

In [ ]:
new_rows = []
for group, group_data in negative.groupby('stay_id'):

    front_rows = pd.DataFrame({'stay_id': [group] * 5, 'starttime': [None] * 5, 'charttime': [None] * 5,
                               'heart_rate': [0] * 5, 'sbp_ni': [0] * 5, 'dbp_ni': [0] * 5,
                               'resp_rate': [0] * 5, 'temperature': [0] * 5, 'spo2': [0] * 5})

    new_rows.extend([front_rows, group_data])

negative1 = pd.concat(new_rows, ignore_index=True)
order = ['subject_id', 'stay_id', 'gender', 'anchor_age', 'charttime', 'starttime', 'heart_rate',  'resp_rate', 'sbp_ni', 'dbp_ni', 'spo2', 'temperature']
negative1 = negative1[order]
negative1

,subject_id,stay_id,gender,anchor_age,charttime,starttime,heart_rate,resp_rate,sbp_ni,dbp_ni,spo2,temperature
0,NaN,30027235,NaN,NaN,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1,NaN,30027235,NaN,NaN,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0
2,NaN,30027235,NaN,NaN,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0
3,NaN,30027235,NaN,NaN,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,NaN,30027235,NaN,NaN,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5774,10678746.0,39967329,M,91.0,2122-01-24 12:00:00,NaN,63.0,20.0,122.0,42.0,100.0,36.4
5775,10678746.0,39967329,M,91.0,2122-01-24 08:00:00,NaN,65.0,18.0,138.0,47.0,100.0,36.6
5776,10678746.0,39967329,M,91.0,2122-01-24 04:00:00,NaN,120.0,8.0,126.0,51.0,100.0,36.6
5777,10678746.0,39967329,M,91.0,2122-01-24 00:00:00,NaN,122.0,14.0,136.0,45.0,100.0,36.6


In [ ]:
negative1['heart_rate'] = negative1['heart_rate'].apply(lambda x:round(x,1))
negative1['sbp_ni'] = negative1['sbp_ni'].apply(lambda x:round(x,1))
negative1['dbp_ni'] = negative1['dbp_ni'].apply(lambda x:round(x,1))
negative1['resp_rate'] = negative1['resp_rate'].apply(lambda x:round(x,1))
negative1['temperature'] = negative1['temperature'].apply(lambda x:round(x,1))
negative1['spo2'] = negative1['spo2'].apply(lambda x:round(x,1))

In [ ]:
# 存放正樣本的dataframe
pos_sampless = pd.DataFrame()

pos_samples = []

order = 1

window_size = 6

for group, group_data in positive1.groupby('stay_id'):
    group_len = len(group_data)

    for i in range(0, group_len, 6):
        pos_sample = group_data.iloc[i:i + window_size]
        pos_sample = pos_sample.copy()
        pos_sample['order'] = order
        pos_samples.append(pos_sample)
        order += 1

pos_sampless = pd.concat(pos_samples)
pos_sampless.reset_index(drop=True, inplace=True)
pos_sampless

,stay_id,starttime,charttime,heart_rate,sbp_ni,dbp_ni,resp_rate,temperature,spo2,order
0,30031755,2124-04-24 17:00:00,2124-04-23 16:00:00,98.0,84.0,47.0,14.0,36.4,95.0,1
1,30031755,2124-04-24 17:00:00,2124-04-23 20:00:00,97.0,75.0,48.0,11.0,36.2,97.0,1
2,30031755,2124-04-24 17:00:00,2124-04-24 00:00:00,87.0,78.0,48.0,16.0,36.0,96.0,1
3,30031755,2124-04-24 17:00:00,2124-04-24 04:00:00,91.0,74.0,44.0,16.0,35.9,89.0,1
4,30031755,2124-04-24 17:00:00,2124-04-24 08:00:00,92.0,81.0,47.0,15.0,35.8,91.0,1
...,...,...,...,...,...,...,...,...,...,...
20767,39992578,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0,3462
20768,39992578,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0,3462
20769,39992578,2187-09-15 10:00:00,2187-09-15 01:00:00,80.0,86.0,37.0,18.0,36.5,100.0,3462
20770,39992578,2187-09-15 10:00:00,2187-09-15 05:00:00,83.0,87.0,35.0,16.0,36.6,100.0,3462


In [ ]:
# 存放負樣本的dataframe
neg_sampless = pd.DataFrame()

neg_samples = []

order = 1

window_size = 6

for group, group_data in negative1.groupby('stay_id'):
    group_len = len(group_data)

    for i in range(group_len - window_size + 1):
        neg_sample = group_data.iloc[i:i + window_size]
        neg_sample = neg_sample.copy()
        neg_sample['order'] = order
        neg_samples.append(neg_sample)
        order += 1

    if group_len < window_size:
        neg_sample = group_data.copy()
        neg_sample['order'] = order
        neg_samples.append(neg_sample)
        order += 1

neg_sampless = pd.concat(neg_samples)
neg_sampless.reset_index(drop=True, inplace=True)
neg_sampless

,subject_id,stay_id,gender,anchor_age,charttime,starttime,heart_rate,resp_rate,sbp_ni,dbp_ni,spo2,temperature,order
0,NaN,30027235,NaN,NaN,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1
1,NaN,30027235,NaN,NaN,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1
2,NaN,30027235,NaN,NaN,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1
3,NaN,30027235,NaN,NaN,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1
4,NaN,30027235,NaN,NaN,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20359,10678746.0,39967329,M,91.0,2122-01-24 12:00:00,NaN,63.0,20.0,122.0,42.0,100.0,36.4,3394
20360,10678746.0,39967329,M,91.0,2122-01-24 08:00:00,NaN,65.0,18.0,138.0,47.0,100.0,36.6,3394
20361,10678746.0,39967329,M,91.0,2122-01-24 04:00:00,NaN,120.0,8.0,126.0,51.0,100.0,36.6,3394
20362,10678746.0,39967329,M,91.0,2122-01-24 00:00:00,NaN,122.0,14.0,136.0,45.0,100.0,36.6,3394


In [ ]:
neg_sampless = neg_sampless.replace(0, np.nan)

In [ ]:
# 計算另一負樣本的各vital signs平均
print(neg_sampless['heart_rate'].dropna().mean())
print(neg_sampless['heart_rate'].dropna().std())
print('-'*30)
print(neg_sampless['sbp_ni'].dropna().mean())
print(neg_sampless['sbp_ni'].dropna().std())
print('-'*30)
print(neg_sampless['dbp_ni'].dropna().mean())
print(neg_sampless['dbp_ni'].dropna().std())
print('-'*30)
print(neg_sampless['resp_rate'].dropna().mean())
print(neg_sampless['resp_rate'].dropna().std())
print('-'*30)
print(neg_sampless['temperature'].dropna().mean())
print(neg_sampless['temperature'].dropna().std())
print('-'*30)
print(neg_sampless['spo2'].dropna().mean())
print(neg_sampless['spo2'].dropna().std())

83.38254703734906
16.468037889701286
------------------------------
119.2912805391744
20.196942433923386
------------------------------
64.13233642235328
14.827472187266206
------------------------------
19.349199663016005
5.303424680167766
------------------------------
36.84712159505752
0.5153456379423281
------------------------------
96.09821679303566
3.283270232723435


In [ ]:
from scipy.stats import levene

# 正負樣本的各vital sign作統計比較
# 變異數
hstatistic, hp_value = levene(pos_sampless['heart_rate'].dropna().tolist(), neg_sampless['heart_rate'].dropna().tolist())
sstatistic, sp_value = levene(pos_sampless['sbp_ni'].dropna().tolist(), neg_sampless['sbp_ni'].dropna().tolist())
dstatistic, dp_value = levene(pos_sampless['dbp_ni'].dropna().tolist(), neg_sampless['dbp_ni'].dropna().tolist())
rstatistic, rp_value = levene(pos_sampless['resp_rate'].dropna().tolist(), neg_sampless['resp_rate'].dropna().tolist())
tstatistic, tp_value = levene(pos_sampless['temperature'].dropna().tolist(), neg_sampless['temperature'].dropna().tolist())
ostatistic, op_value = levene(pos_sampless['spo2'].dropna().tolist(), neg_sampless['spo2'].dropna().tolist())


print('統計量：%.2f'%hstatistic)
print('p-value:%.5f'%hp_value)
print('-'*30)
print('統計量：%.2f'%sstatistic)
print('p-value:%.5f'%sp_value)
print('-'*30)
print('統計量：%.2f'%dstatistic)
print('p-value:%.5f'%dp_value)
print('-'*30)
print('統計量：%.2f'%rstatistic)
print('p-value:%.5f'%rp_value)
print('-'*30)
print('統計量：%.2f'%tstatistic)
print('p-value:%.5f'%tp_value)
print('-'*30)
print('統計量：%.2f'%ostatistic)
print('p-value:%.5f'%op_value)
print('-'*30)

統計量：6233.14
p-value:0.00000
------------------------------
統計量：5803.90
p-value:0.00000
------------------------------
統計量：4354.96
p-value:0.00000
------------------------------
統計量：4256.26
p-value:0.00000
------------------------------
統計量：13165.87
p-value:0.00000
------------------------------
統計量：12353.02
p-value:0.00000
------------------------------


In [ ]:
from scipy.stats import ttest_ind

# 獨立t test

print(ttest_ind(pos_sampless['heart_rate'].dropna().tolist(), neg_sampless['heart_rate'].dropna().tolist(), equal_var=False, alternative='two-sided'))
print('-'*30)
print(ttest_ind(pos_sampless['sbp_ni'].dropna().tolist(), neg_sampless['sbp_ni'].dropna().tolist(), equal_var=False, alternative='two-sided'))
print('-'*30)
print(ttest_ind(pos_sampless['dbp_ni'].dropna().tolist(), neg_sampless['dbp_ni'].dropna().tolist(), equal_var=False, alternative='two-sided'))
print('-'*30)
print(ttest_ind(pos_sampless['resp_rate'].dropna().tolist(), neg_sampless['resp_rate'].dropna().tolist(), equal_var=False, alternative='two-sided'))
print('-'*30)
print(ttest_ind(pos_sampless['temperature'].dropna().tolist(), neg_sampless['temperature'].dropna().tolist(), equal_var=False, alternative='two-sided'))
print('-'*30)
print(ttest_ind(pos_sampless['spo2'].dropna().tolist(), neg_sampless['spo2'].dropna().tolist(), equal_var=False, alternative='two-sided'))

Ttest_indResult(statistic=-102.10324797681358, pvalue=0.0)
------------------------------
Ttest_indResult(statistic=-157.20011966178689, pvalue=0.0)
------------------------------
Ttest_indResult(statistic=-133.11659657104653, pvalue=0.0)
------------------------------
Ttest_indResult(statistic=-89.74378895163032, pvalue=0.0)
------------------------------
Ttest_indResult(statistic=-148.07232785391605, pvalue=0.0)
------------------------------
Ttest_indResult(statistic=-148.37409598117043, pvalue=0.0)
